#### 3 The Mariners.


(3) Using the MLB averages is not the best way to replace nan values. A Markov chain should have rows sum to 1. However by replace these rows with the mlb average our Matrix no longer has the Markov Properties. One possible fix is to just replace these values with 0 (which is what I did in part 1). Alternatively we could update it with their percentage for single, double, or triple with a different number of outs (and if that doesn't work a different number of base runners). Another idea would be to just update with their averages for a single, double, or triple overall.


(4-6) I'm rather confused how the file runs.csv has negative values, since you can't have negative runs in baseball. However the results were mostly as expected:

(Lineup1, Lineup2) = (7.64, 8.55)

We expected Lineup 2 to have a higher average than Lineup 1. However at first glance these numbers seem inflated, an average of 8.56 runs per game would be amazing in the game of baseball!

#### PART 1 CODE

In [245]:
import numpy as np
import csv

times = []
b = []
 
# Transition Matrix: There are 25 Possible States: 8 Possible variations of runners on base with possible outs of 0, 1, or 2. Giving 24 states.
# There is one additional state for 3 outs (runners on base will always be 000 since the inning is over)
G1 = np.zeros((24,25)) # Gorden Transition
G2 = np.zeros((24,25)) # Gamel Transition

def state(s):
    s = s.split(" ")
    r = s[0] # base runners

    # return state 25, if there are 3 outs
    if r[0] == '3':
        return 24

    n_outs = int(s[1])
    # value for base runners element of [0,7]
    b_val = int(r[0]) + int(r[1])*2 + int(r[2])*4

    # 0 outs : (0,7), 1 out : (8,15), 2 outs : (16,23)
    return b_val + (8*n_outs)

# G - Play Number, Batter (Gorden or Gamel), Base Runner (011 Second and Third) Number of Outs, Base Runner Number of Outs : After Bat
with open("G.csv", mode="r") as f:
    lines = csv.reader(f)

    for l in lines:

        # Gordon
        if l[1] == "Gordon":
            G1[state(l[2])][state(l[3])] += 1

        # Gamel
        elif l[1] == "Gamel":
            G2[state(l[2])][state(l[3])] += 1

        else:
            print("Invalid: ", l[1])

Invalid:  batter


In [246]:

MLB_AVG= np.zeros((24,25)) # MLB AVG Transition
# G - Play Number, Batter (Gorden or Gamel), Base Runner (011 Second and Third) Number of Outs, Base Runner Number of Outs : After Bat
with open("MLBAVG.csv", mode="r") as f:
    lines = csv.reader(f)
    r_num=-2

    for l in lines:
        try:
            r_num+=1
            MLB_AVG[r_num] = [float(l[i]) for i in range(1,26)]
        except Exception:
            print("Exception: ", l)

Exception:  ['', '000 0', '000 1', '000 2', '001 0', '001 1', '001 2', '010 0', '010 1', '010 2', '011 0', '011 1', '011 2', '100 0', '100 1', '100 2', '101 0', '101 1', '101 2', '110 0', '110 1', '110 2', '111 0', '111 1', '111 2', '3']


In [248]:
# Normalized G1
n1 = [[G1[i][j] / sum(G1[i]) if sum(G1[i]) != 0 else 0 for j in range(25)] for i in range(24)]
n1 = np.array(n1)

# Normalized G2
n2 = [[G2[i][j] / sum(G2[i]) if sum(G2[i]) != 0 else 0 for j in range(25)] for i in range(24)]
n2 = np.array(n2)

In [249]:
print("First Row of Transition Matrix 1:")
print(n1[0])

print()
print("First Row of Transition Matrix 2:")
print(n2[0])

First Row of Transition Matrix 1:
[0.00490196 0.25       0.06372549 0.         0.01960784 0.
 0.         0.         0.66176471 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.        ]

First Row of Transition Matrix 2:
[0.01219512 0.29268293 0.08536585 0.         0.03658537 0.
 0.         0.         0.57317073 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.        ]


#### PART 2 CODE

In [ ]:
# Part 2 - Replace 0 values with MLB Average
# Normalized G1
n1 = [[G1[i][j] / sum(G1[i]) if sum(G1[i]) != 0 else MLB_AVG[i][j] for j in range(25)] for i in range(24)]
n1 = np.array(n1)

# Normalized G2
n2 = [[G2[i][j] / sum(G2[i]) if sum(G2[i]) != 0 else MLB_AVG[i][j] for j in range(25)] for i in range(24)]
n2 = np.array(n2)

In [182]:
print("First Row of Transition Matrix 1:")
print(n1[0])

print()
print("First Row of Transition Matrix 2:")
print(n2[0])

First Row of Transition Matrix 1:
[0.00490196 0.25       0.06372549 0.         0.01960784 0.
 0.         0.         0.66176471 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.        ]

First Row of Transition Matrix 2:
[0.01219512 0.29268293 0.08536585 0.         0.03658537 0.
 0.         0.         0.57317073 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.        ]


#### PART 4-6 CODE

In [183]:
RUNS = np.zeros((24,25)) # RUNS Matrix
with open("runs.csv", mode="r") as f:
    lines = csv.reader(f)
    r_num=-2

    for l in lines:
        try:
            r_num+=1
            RUNS[r_num] = [float(l[i]) for i in range(1,26)]
        except Exception:
            pass

Exception:  ['', '000 0', '000 1', '000 2', '001 0', '001 1', '001 2', '010 0', '010 1', '010 2', '011 0', '011 1', '011 2', '100 0', '100 1', '100 2', '101 0', '101 1', '101 2', '110 0', '110 1', '110 2', '111 0', '111 1', '111 2', '']


In [321]:
# lineup - list of transition matrices
# total_games - number of games to simulate
def batting_sim(lineup, total_games=162):
    num_runs = 0
    
    for games in range(total_games): # simulate each game
        curr_batter = 0

        for inning in range(9):  # 9 innings
            state = 0 # reset number of outs
            nstate = 0

            # print(str(inning) + ": ", num_runs)

            while state != 24: # while not 3 outs
                r = lineup[curr_batter][state]
                nstate = next_state(r)

                # Next Batter
                if curr_batter == 8: curr_batter = 0
                else: curr_batter += 1

                # Update the Number of runs after state transition
                num_runs += RUNS[state][nstate]
                state = nstate

    return num_runs/total_games

# Returns the Next State given the Distribution
def next_state(r):
    p = np.random.rand(1)[0] # Get Probability
    r = [sum(r[:i]) for i in range(1,26)] # CDF

    for i in range(25):
        if p <= round(r[i],2):
            return i

# Compares Lineups
def compare(lineup1, lineup2, N=100):
    l1 = sum(batting_sim(lineup1) for i in range(N))/N # Simulate N times and average the number of runs scored/drive over the N seasons
    l2 = sum(batting_sim(lineup2) for i in range(N))/N # Simulate N times and average the number of runs scored/drive over the N seasons

    return((l1,l2))

In [319]:
# Set Up Lineups

lineup1, lineup2 = [0 for i in range(9)], [0 for i in range(9)] # initialize lists
def get_matrix(path):
    T = np.zeros((24,25))

    with open(path, mode="r") as f:
        lines = csv.reader(f)
        r_num=-2
        for l in lines:
            try:
                r_num+=1
                T[r_num] = [float(l[i]) for i in range(1,26)]
            except Exception: 
                pass

    for i in range(24):
        if sum(T[i]) == 0:
            T[i] = MLB_AVG[i]
        else:
            T[i] = T[i]/sum(T[i])

                

    return np.array(T)

lineup1 = [get_matrix("mhaniger.csv"),get_matrix("jsegura.csv"),get_matrix("rcano.csv"),get_matrix("ncruz.csv"),get_matrix("kseager.csv"),get_matrix("rhealy.csv"), 
    get_matrix("dspan.csv"), get_matrix("mzunino.csv"), n1]

lineup2 = [get_matrix("mhaniger.csv"),get_matrix("jsegura.csv"),get_matrix("rcano.csv"),get_matrix("ncruz.csv"),get_matrix("dspan.csv"),get_matrix("kseager.csv"), n2,
    get_matrix("mzunino.csv"), get_matrix("rhealy.csv")]

In [322]:
compare(lineup1, lineup2)

(7.640555555555558, 8.552654320987651)